In [32]:
# Import the dependencies.
import pandas as pd

#Numpy is needed for random function
import numpy as np

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the requests library.  Used to make HTTP Requests
import requests

# Import time library - Used to sleep, convert between time representations, has timezone attributes
import time

# Import datetime library - Used to do date/time conversions from UTC format and such
from datetime import datetime



In [33]:
# OpenWeatherMap API Call Setup - Get API Key and Set URL - Weather Data
from config import weather_api_key

# Starting URL for Weather Map API Call.
# Add the units=Imperial to set temperature to Farenheit
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [34]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats,lngs)

# Results are zipped
# The zip object packs each pair of lats and lngs having the same index in their 
# respective array into a tuple. If there are 1,500 latitudes and longitudes, 
# there will be 1,500 tuples of paired latitudes and longitudes, where each 
# latitude and longitude in a tuple can be accessed by the index of 0 and 1, respectively.

In [35]:
# Add the latitudes and longitudes to a list
# You can only unzip a zipped tuple once before it is removed from the computer's memory. 
# Make sure you unzip the latitudes and longitudes into the coordinates list before moving on.
coordinates = list(lat_lngs)

In [36]:
# Use the tuple() function to display the latitude and longitude combination for coordinate in coordinates
# create a list for the cities
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique then add it to the cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

748

In [37]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval       ")
print("_______________________________")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):

    #Group cities in sets of 50 for logging purposes
    #Pause for 60 seconds to all use of the free version of the API (only allows 50 calls per minute)
    if (i % 50) == 0 and (i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Rn an API request for the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]

        # The WEATHER info is a Dictionary so we need to use an Index to access the values. 
        city_current_weather = city_weather["weather"][0]["description"]
        
        #Convert the date to ISO standard - This is not needed for this exercise but left in for future example/research
        # city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        #Append the city information into city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country":city_country,
                          "Current Description": city_current_weather})
    #If an error occurs, skip the city.
    except:
        print("city not found. Skipping...")
        # Generally, it isn't good coding practice to add 
        # the pass statement to the except block. Ideally, we want to handle or catch each error 
        # as it happens and do something specific (e.g., add another try block or print out the error).
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval       
_______________________________
Processing Record 1 of Set 1 | taolanaro
city not found. Skipping...
Processing Record 2 of Set 1 | katsuura
Processing Record 3 of Set 1 | amderma
city not found. Skipping...
Processing Record 4 of Set 1 | ixtapa
Processing Record 5 of Set 1 | hambantota
Processing Record 6 of Set 1 | husavik
Processing Record 7 of Set 1 | wanning
Processing Record 8 of Set 1 | boguchany
Processing Record 9 of Set 1 | pauini
Processing Record 10 of Set 1 | ouesso
Processing Record 11 of Set 1 | victoria
Processing Record 12 of Set 1 | mayo
Processing Record 13 of Set 1 | busselton
Processing Record 14 of Set 1 | havre
Processing Record 15 of Set 1 | punta arenas
Processing Record 16 of Set 1 | bluff
Processing Record 17 of Set 1 | port hardy
Processing Record 18 of Set 1 | lebu
Processing Record 19 of Set 1 | ancud
Processing Record 20 of Set 1 | luena
Processing Record 21 of Set 1 | sao felipe
Processing Record 22 of Set 1 | ketchikan
Pr

In [38]:
# Convert the list/array of dictionaries to a Pandas Dataframe
city_data_df = pd.DataFrame(city_data)
new_column_order=["City", "Country","Lat","Lng", "Max Temp", "Humidity","Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]

# Check to make sure the data is as it should be before exporting
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Katsuura,JP,35.1333,140.3000,37.11,96,100,25.28,overcast clouds
1,Ixtapa,MX,20.7000,-105.2000,80.58,24,0,3.04,clear sky
2,Hambantota,LK,6.1241,81.1185,78.24,78,69,1.70,moderate rain
3,Husavik,IS,66.0449,-17.3389,28.31,83,81,13.09,broken clouds
4,Wanning,CN,18.8003,110.3967,69.60,93,25,5.64,scattered clouds
5,Boguchany,RU,58.3814,97.4531,-13.68,100,97,2.98,overcast clouds
6,Pauini,BR,-7.7136,-66.9764,82.56,67,99,5.03,overcast clouds
7,Ouesso,CG,1.6136,16.0517,88.18,20,18,0.81,few clouds
8,Victoria,HK,22.2855,114.1577,64.45,87,100,11.99,overcast clouds
9,Mayo,US,38.8876,-76.5119,59.34,43,0,12.66,clear sky


In [39]:
# Create the Output file (CSV
output_data_file = "WeatherPy_Database.csv"

#Export the City_Data into a CSV
# The last line in the code block will export the DataFrame to a CSV file, 
# with the index label (or column A) header as "City_ID." If we ever need 
# to export the CSV file to a DataFrame, that header will be present in the DataFrame.
city_data_df.to_csv(output_data_file, index_label="City_ID")